This notebook is designed to run in a IBM Watson Studio default runtime (NOT the Watson Studio Apache Spark Runtime as the default runtime with 1 vCPU is free of charge). Therefore, we install Apache Spark in local mode for test purposes only. Please don't use it in production.

In case you are facing issues, please read the following two documents first:

https://github.com/IBM/skillsnetwork/wiki/Environment-Setup

https://github.com/IBM/skillsnetwork/wiki/FAQ

Then, please feel free to ask:

https://coursera.org/learn/machine-learning-big-data-apache-spark/discussions/all

Please make sure to follow the guidelines before asking a question:

https://github.com/IBM/skillsnetwork/wiki/FAQ#im-feeling-lost-and-confused-please-help-me


If running outside Watson Studio, this should work as well. In case you are running in an Apache Spark context outside Watson Studio, please remove the Apache Spark setup in the first notebook cells.

In [1]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown('# <span style="color:red">'+string+'</span>'))


if ('sc' in locals() or 'sc' in globals()):
    printmd('<<<<<!!!!! It seems that you are running in a IBM Watson Studio Apache Spark Notebook. Please run it in an IBM Watson Studio Default Runtime (without Apache Spark) !!!!!>>>>>')


In [2]:
!pip install pyspark==2.4.5

     |████████████████████████████████| 217.8MB 149kB/s  eta 0:00:01     |███████████████████████████▋    | 187.6MB 15.3MB/s eta 0:00:02
     |████████████████████████████████| 204kB 36.2MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/bf/db/04/61d66a5939364e756eb1c1be4ec5bdce6e04047fc7929a3c3c
Successfully built pyspark


In [3]:
try:
    from pyspark import SparkContext, SparkConf
    from pyspark.sql import SparkSession
except ImportError as e:
    printmd('<<<<<!!!!! Please restart your kernel after installing Apache Spark !!!!!>>>>>')

In [4]:
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

spark = SparkSession \
    .builder \
    .getOrCreate()

This is the third assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Again, please insert to code to your ApacheCouchDB based Cloudant instance below using the "Insert Code" function of Watson Studio (you've done this in Assignment 1 and 2 before)

Done, just execute all cells one after the other and you are done - just note that in the last one you must update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"


In [5]:
!wget https://github.com/IBM/skillsnetwork/raw/master/coursera_ml/shake.parquet

--2020-04-27 17:49:49--  https://github.com/IBM/skillsnetwork/raw/master/coursera_ml/shake.parquet
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ml/shake.parquet [following]
--2020-04-27 17:49:50--  https://raw.githubusercontent.com/IBM/skillsnetwork/master/coursera_ml/shake.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74727 (73K) [application/octet-stream]
Saving to: ‘shake.parquet.1’

100%[======================================>] 74,727      --.-K/s   in 0.004s  

2020-04-27 17:49:50 (16.3 MB/s) - ‘shake.parquet.1’ saved [74727/74727]



Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [6]:
df=spark.read.parquet('shake.parquet')

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+---------+-----+-----+-----+
|CLASS| SENSORID|    X|    Y|    Z|
+-----+---------+-----+-----+-----+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|
|    2|aUniqueID| 0.03| 0.03| 0.03|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|
|    2| 12345678| -0.1| -0.1| -0.1|
|    2| 12345678|-0.15|-0.15|-0.15|
|    2| 12345678| 0.47| 0.47| 0.47|
|    2| 12345678|-0.06|-0.06|-0.06|
|    2| 12345678|-0.09|-0.09|-0.09|
|    2| 12345678| 0.21| 0.21| 0.21|
|    2| 12345678|-0.08|-0.08|-0.08|
|    2| 12345678| 0.44| 0.44| 0.44|
|    2|    gholi| 0.76| 0.76| 0.76|
|    2|    gholi| 1.62| 1.62| 1.62|
|    2|    gholi| 5.81| 5.81| 5.81|
|    2| bcbcbcbc| 0.58| 0.58| 0.58|
|    2| bcbcbcbc|-8.24|-8.24|-8.24|
|    2| bcbcbcbc|-0.45|-0.45|-0.45|
|    2| bcbcbcbc| 1.03| 1.03| 1.03|
|    2|aUniqueID|-0.05|-0.05|-0.05|
|    2| qqqqqqqq|-0.44|-0.44|-0.44|
+-----+---------+-----+-----+-----+
only showing top 20 rows



Let’s check if we have balanced classes – this means that we have roughly the same number of examples for each class we want to predict. This is important for classification but also helpful for clustering

In [7]:
spark.sql("SELECT count(class), class from df group by class").show()

+------------+-----+
|count(class)|class|
+------------+-----+
|          14|    0|
|         119|    1|
|        9750|    2|
+------------+-----+



Let's create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [8]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")

Please insatiate a clustering algorithm from the SparkML package and assign it to the clust variable. Here we don’t need to take care of the “CLASS” column since we are in unsupervised learning mode – so let’s pretend to not even have the “CLASS” column for now – but it will become very handy later in assessing the clustering performance. PLEASE NOTE – IN REAL-WORLD SCENARIOS THERE IS NO CLASS COLUMN – THEREFORE YOU CAN’T ASSESS CLASSIFICATION PERFORMANCE USING THIS COLUMN 



In [9]:
from pyspark.ml.clustering import KMeans

clust = KMeans().setK(2).setSeed(1)



In [10]:


from pyspark.ml.clustering import GaussianMixture

clust = GaussianMixture().setK(2).setSeed(538009335)

Let’s train...


In [11]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, clust])
model = pipeline.fit(df)

...and evaluate...

In [12]:
prediction = model.transform(df)
prediction.show()

+-----+---------+-----+-----+-----+-------------------+----------+--------------------+
|CLASS| SENSORID|    X|    Y|    Z|           features|prediction|         probability|
+-----+---------+-----+-----+-----+-------------------+----------+--------------------+
|    2| qqqqqqqq| 0.12| 0.12| 0.12|   [0.12,0.12,0.12]|         1|[0.01003342959175...|
|    2|aUniqueID| 0.03| 0.03| 0.03|   [0.03,0.03,0.03]|         1|[0.00871793613116...|
|    2| qqqqqqqq|-3.84|-3.84|-3.84|[-3.84,-3.84,-3.84]|         0|[0.99999999999970...|
|    2| 12345678| -0.1| -0.1| -0.1|   [-0.1,-0.1,-0.1]|         1|[0.00906342315452...|
|    2| 12345678|-0.15|-0.15|-0.15|[-0.15,-0.15,-0.15]|         1|[0.00992723341782...|
|    2| 12345678| 0.47| 0.47| 0.47|   [0.47,0.47,0.47]|         1|[0.06148666840828...|
|    2| 12345678|-0.06|-0.06|-0.06|[-0.06,-0.06,-0.06]|         1|[0.00868702964539...|
|    2| 12345678|-0.09|-0.09|-0.09|[-0.09,-0.09,-0.09]|         1|[0.00894510576286...|
|    2| 12345678| 0.21| 0.21| 0.

In [13]:
prediction.createOrReplaceTempView('prediction')
spark.sql('''
select max(correct)/max(total) as accuracy from (

    select sum(correct) as correct, count(correct) as total from (
        select case when class != prediction then 1 else 0 end as correct from prediction 
    ) 
    
    union
    
    select sum(correct) as correct, count(correct) as total from (
        select case when class = prediction then 1 else 0 end as correct from prediction 
    ) 
)
''').rdd.map(lambda row: row.accuracy).collect()[0]

0.9879591217241728

If you reached at least 55% of accuracy you are fine to submit your predictions to the grader. Otherwise please experiment with parameters setting to your clustering algorithm, use a different algorithm or just re-record your data and try to obtain. In case you are stuck, please use the Coursera Discussion Forum.

In [14]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2020-04-27 17:51:05--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2020-04-27 17:51:05 (17.0 MB/s) - ‘rklib.py’ saved [2289/2289]



In [15]:
data = spark.sql("""
    select x,y,z,
        case 
            when class = 0 and prediction = 1 then 'r' 
            when class = 0 and prediction = 0 then 'y' 
            when class = 1 and prediction = 0 then 'g'
            when class = 1 and prediction = 1 then 'b'
            else 'w' end as c 
        from prediction 
""").sample(False, 0.2, 42).rdd.map(lambda row: (row.x, row.y, row.z, row.c)).collect()


In [16]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np



fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')


for x,y,z,c in data:
    None
    ax.scatter(x, y, z, c=c)

    
for c in model.stages[1].clusterCenters():
    ax.scatter(c[0], c[1], c[2], marker="^")

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()



AttributeError: 'GaussianMixtureModel' object has no attribute 'clusterCenters'

In [140]:
prediction= prediction.repartition(1)
prediction.write.parquet('a2_m3.parquet')

In [141]:
!zip -r a2_m3.parquet.zip a2_m3.parquet

  adding: a2_m3.parquet/ (stored 0%)
  adding: a2_m3.parquet/part-00000-0119ce4d-d78c-4830-851b-c0f9edf15aff.snappy.parquet (deflated 20%)
  adding: a2_m3.parquet/_SUCCESS (stored 0%)
  adding: a2_m3.parquet/._SUCCESS.crc (stored 0%)
  adding: a2_m3.parquet/.part-00000-0119ce4d-d78c-4830-851b-c0f9edf15aff.snappy.parquet.crc (stored 0%)


In [142]:
!base64 a2_m3.parquet.zip > a2_m3.parquet.zip.base64

In [145]:
from rklib import submit
key = "pPfm62VXEeiJOBL0dhxPkA"
part = "EOTMs"
email = "romeo.kienzler@ch.ibm.com"
secret = "aLGsRJbEKITw9Cr3"

with open('a2_m3.parquet.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
